<a href="https://colab.research.google.com/github/Teapack1/Notebooks-Scripts/blob/main/FT_w_Transformers_TRL_SFT_PEFT_QLoRA_generic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# `transformers` meets `bitsandbytes` for democratzing Large Language Models (LLMs) through 4bit quantization

<center>
<img src="https://github.com/huggingface/blog/blob/main/assets/96_hf_bitsandbytes_integration/Thumbnail_blue.png?raw=true" alt="drawing" width="700" class="center"/>
</center>

Welcome to this notebook that goes through the recent `bitsandbytes` integration that includes the work from XXX that introduces no performance degradation 4bit quantization techniques, for democratizing LLMs inference and training.

In this notebook, we will learn together how to load a large model in 4bit (`gpt-neo-x-20b`) and train it using Google Colab and PEFT library from Hugging Face 🤗.

[In the general usage notebook](https://colab.research.google.com/drive/1ge2F1QSK8Q7h0hn3YKuBCOAS0bK8E0wf?usp=sharing), you can learn how to propely load a model in 4bit with all its variants.

If you liked the previous work for integrating [*LLM.int8*](https://arxiv.org/abs/2208.07339), you can have a look at the [introduction blogpost](https://huggingface.co/blog/hf-bitsandbytes-integration) to lean more about that quantization method.


# 1) Dependancies, Check-in

In [ ]:
!pip uninstall -y transformers
!pip install git+https://github.com/huggingface/transformers

Found existing installation: transformers 4.35.2
Uninstalling transformers-4.35.2:
  Successfully uninstalled transformers-4.35.2
  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-gpvb4fx8
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-gpvb4fx8
  Resolved https://github.com/huggingface/transformers to commit a75a6c9315fb8fe23934a43028be6fb8a191351d
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.38.0.dev0-py3-none-any.whl size=8505490 sha256=d155e496ff61eae3413d0a557b13b489ef8a61918d09736069d0b46e6a53cc44
  Stored in directory: /tmp/pip-ephem-wheel-cache-e2zj7cg6/wheels/c0/14/d6/6c9a5582d2ac191ec0a483be151a4495fe1eb2a6706ca49f1b
Successfully built transformers


In [ ]:
!pip install -q bitsandbytes
!pip install git+https://github.com/huggingface/transformers
!pip install -q peft
!pip install -q accelerate
%pip install -U datasets
!pip install -q huggingface-hub
!pip install -q trl
!pip install -q bitsandbytes

In [ ]:
!pip install flash-attn --no-build-isolation

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 16.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 14.6 MB/s eta 0:00:00
  Created wheel for flash-attn: filename=flash_attn-2.5.3-cp310-cp310-linux_x86_64.whl size=120817842 sha256=26616addbd1356f9cdf3b347bed9dfc7c47bf9e1465b8b3cacbade5069e79555
  Stored in directory: /root/.cache/pip/wheels/a1/ff/66/7f33fff3f6367fc46a010fd8f32ee4c3958b9b0730e61ec4b2
Successfully built flash-attn


In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
model_id = "microsoft/phi-2"
trained_lora = "LoRA-phi-2-LexFridman"
new_model = "phi-2-LexFridman"
#dataset="/kaggle/input/shorts-yt/dataset.csv"

In [ ]:
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"

device_map = {"": device}
device_map = "auto"

In [ ]:
from pynvml import *

def print_gpu_utilization():
    nvmlInit()
    handle = nvmlDeviceGetHandleByIndex(0)
    info = nvmlDeviceGetMemoryInfo(handle)
    print(f"GPU memory occupied: {info.used//1024**2} MB.")

def print_summary(result):
    print(f"Time: {result.metrics['train_runtime']:.2f}")
    print(f"Samples/second: {result.metrics['train_samples_per_second']:.2f}")
    print_gpu_utilization()

print_gpu_utilization()

# 2) Handle the Dataset

## 2.a Prompt formats Examples
Setup desirable training prompt format.

#### Create ChatML Prompt
In the following function we'll be merging our `prompt` and `response` columns by creating the following template:

In [ ]:
# we need to reformat the data in teh ChatML format.

def create_prompt(input,response)->str:
    return f"<|im_start|>user\n{input}<|im_end|>\n<|im_start|>assistant\n{response}<|im_end|>\n"

#### Create Instruct Prompt
In the following function we'll be merging our `prompt` and `response` columns by creating the following template:

```
<s>### Instruction:
Use the provided input to create an instruction that could have been used to generate the response with an LLM.

### Input:
{input}

### Response:
{response}</s>
```

In [ ]:
# Prompt formatting - optional -> enable in the Trainer parameters bellow.

def create_prompt(sample):
  """
  Update the prompt template:
  Combine both the prompt and input into a single column.

  """
  bos_token = "<s>"
  original_system_message = "Below is an instruction that describes a task. Write a response that appropriately completes the request."
  system_message = "Use the provided input to create an instruction that could have been used to generate the response with an LLM."
  response = sample["prompt"].replace(original_system_message, "").replace("\n\n### Instruction\n", "").replace("\n### Response\n", "").strip()
  input = sample["response"]
  eos_token = "</s>"

  full_prompt = ""
  full_prompt += bos_token
  full_prompt += "### Instruction:"
  full_prompt += "\n" + system_message
  full_prompt += "\n\n### Input:"
  full_prompt += "\n" + input
  full_prompt += "\n\n### Response:"
  full_prompt += "\n" + response
  full_prompt += eos_token

  return full_prompt

In [ ]:
# Function to transform the row into desired format
def create_prompt(row):
    question = row['Context']
    answer = row['Response']
    formatted_string = f"[INST] {question} [/INST] {answer} "
    return formatted_string

# Apply the function to each row of the dataframe
df['Formatted'] = df.apply(create_prompt, axis=1)

## 2.b Load Custom Dataset

In [ ]:
import pandas as pd
from datasets import load_dataset, Dataset
# Convert to DataFrame
columns = ["speaker", "content", "title", "start", "end"]

df = pd.read_csv(dataset, header=None, names=columns)

# Display the first few rows of the DataFrame
df.head(2)



In [ ]:
df['speaker'] = df['speaker'].replace({'user': '###Chris: ', 'assistant': '###Host: '})

df["formatted_text"] = df["speaker"].astype(str) + df["content"].astype(str)

df = df.groupby('title').agg({'formatted_text':'\n'.join}).reset_index()

def format_entry(row):
  title = row["title"]
  context = "You are ###Chris a clever, confident and strong self made man. Your life mission is to help other men achieve self realisation. You provide advice and encouragement to young men, giving them raw, uncensored and real life lessons. Your guidance leads through success and fulfilment in fitness, money and sex life. You are ###Chris a podcast host and you interview people like ###Host who got successfull in fitness, money and sex life. You respond in a way you talk with podcast ###Host, you provide honest, uncensored, clever and practical answers to in fitness, money and sex life related questions. Talk as you do with the podcast ###Host and use the knowledge from the podcast conversations."
  content = row["formatted_text"]
  output = f"###System: {context} This a conversation on topic of {title}. {content}"
  return output

df['text'] = df.apply(format_entry, axis=1)

df.head()

In [ ]:
df.text.iloc[1]

In [ ]:
# If you want to save the new dataframe to a CSV file:
# df.to_csv('formatted_data.csv', index=False)

In [ ]:
processed_dataset = Dataset.from_pandas(df)

# HF Dataset

In [ ]:
import pandas as pd
from datasets import load_dataset, Dataset

In [ ]:
dataset = load_dataset("RamAnanth1/lex-fridman-podcasts", split="train")

df = pd.DataFrame(dataset)

# Display the first few rows of the DataFrame
df.head(2)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


,captions,title
0,"As part of MIT course 6S099, Artificial Gener...",Max Tegmark: Life 3.0 | Lex Fridman Podcast #1
1,As part of MIT course 6S099 on artificial gen...,Christof Koch: Consciousness | Lex Fridman Pod...


In [ ]:
df = df.drop(columns="title")
df = df.rename(columns={"captions" : "text"})
df.head(2)

,text
0,"As part of MIT course 6S099, Artificial Gener..."
1,As part of MIT course 6S099 on artificial gen...


In [ ]:
processed_dataset = Dataset.from_pandas(df)

In [ ]:
processed_dataset

Dataset({
    features: ['text'],
    num_rows: 319
})

## 2.c Load HF dataset and Apply Chat Formating

In [ ]:
from datasets import load_dataset, DatasetDict

# based on config
raw_datasets = load_dataset("HuggingFaceH4/ultrachat_200k")

KeyboardInterrupt: 

In [ ]:
# remove this when done debugging
indices = range(0,100)

dataset_dict = {"train": raw_datasets["train_sft"].select(indices),
                "test": raw_datasets["test_sft"].select(indices)}

raw_datasets = DatasetDict(dataset_dict)
raw_datasets

In [ ]:
example = raw_datasets["train"][0]
print(example.keys())

In [ ]:
messages = example["messages"]
for message in messages:
  role = message["role"]
  content = message["content"]
  print('{0:20}:  {1}'.format(role, content))

In [ ]:
from transformers import AutoTokenizer

model_id = "mistralai/Mistral-7B-v0.1"

tokenizer = AutoTokenizer.from_pretrained(model_id)

# set pad_token_id equal to the eos_token_id if not set
if tokenizer.pad_token_id is None:
  tokenizer.pad_token_id = tokenizer.eos_token_id

# Set reasonable default for models without max length
if tokenizer.model_max_length > 100_000:
  tokenizer.model_max_length = 2048

# Set chat template
DEFAULT_CHAT_TEMPLATE = "{% for message in messages %}\n{% if message['role'] == 'user' %}\n{{ '<|user|>\n' + message['content'] + eos_token }}\n{% elif message['role'] == 'system' %}\n{{ '<|system|>\n' + message['content'] + eos_token }}\n{% elif message['role'] == 'assistant' %}\n{{ '<|assistant|>\n'  + message['content'] + eos_token }}\n{% endif %}\n{% if loop.last and add_generation_prompt %}\n{{ '<|assistant|>' }}\n{% endif %}\n{% endfor %}"
tokenizer.chat_template = DEFAULT_CHAT_TEMPLATE

In [ ]:
import re
import random
from multiprocessing import cpu_count

def apply_chat_template(example, tokenizer):
    messages = example["messages"]
    # We add an empty system message if there is none
    if messages[0]["role"] != "system":
        messages.insert(0, {"role": "system", "content": ""})
    example["text"] = tokenizer.apply_chat_template(messages, tokenize=False)

    return example

column_names = list(raw_datasets["train"].features)
raw_datasets = raw_datasets.map(apply_chat_template,
                                num_proc=cpu_count(),
                                fn_kwargs={"tokenizer": tokenizer},
                                remove_columns=column_names,
                                desc="Applying chat template",)

# create the splits
train_dataset = raw_datasets["train"]
eval_dataset = raw_datasets["test"]

for index in random.sample(range(len(raw_datasets["train"])), 3):
  print(f"Sample {index} of the processed training set:\n\n{raw_datasets['train'][index]['text']}")

In [ ]:
raw_datasets

# 2.d Plot Dataset Input Lengths

In [ ]:
def tokenize_prompts(prompt):
    return tokenizer(create_prompt(prompt))

tokenized_train_dataset = instruct_tune_dataset["train"].map(tokenize_prompts)
tokenized_val_dataset = instruct_tune_dataset["test"].map(tokenize_prompts)

In [ ]:
def plot_data_lengths(tokenized_train_dataset, tokenized_val_dataset):
    lengths = [len(x['input_ids']) for x in tokenized_train_dataset]
    lengths += [len(x['input_ids']) for x in tokenized_val_dataset]
    print(len(lengths))

    # Plotting the histogram
    plt.figure(figsize=(10, 6))
    plt.hist(lengths, bins=50, alpha=0.7, color='blue')
    plt.xlabel('Length of input_ids')
    plt.ylabel('Frequency')
    plt.title('Distribution of Lengths of input_ids')
    plt.xlim([0, 2048])
    plt.show()


plot_data_lengths(tokenized_train_dataset, tokenized_val_dataset)

# 3) Loading the Quantized Base Model & Tokenizer

In [ ]:
"""
See Models parameters, if you want to...
"""

def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [ ]:
"""
Load tokenizer of the base model.
"""

from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(
    model_id,
    use_fast=True, # Use Rust-based tokenizer, if availiable
    #trust_remote_code=True #Run custom models from HUB in their own modeling files -> runs their code on the local machine.
)

#if tokenizer.pad_token_id is None:
tokenizer.pad_token_id = tokenizer.eos_token_id

tokenizer.padding_side = "right"

# Set chat template
#DEFAULT_CHAT_TEMPLATE = "{% for message in messages %}\n{% if message['role'] == 'user' %}\n{{ '<|user|>\n' + message['content'] + eos_token }}\n{% elif message['role'] == 'system' %}\n{{ '<|system|>\n' + message['content'] + eos_token }}\n{% elif message['role'] == 'assistant' %}\n{{ '<|assistant|>\n'  + message['content'] + eos_token }}\n{% endif %}\n{% if loop.last and add_generation_prompt %}\n{{ '<|assistant|>' }}\n{% endif %}\n{% endfor %}"
#tokenizer.chat_template = DEFAULT_CHAT_TEMPLATE

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
import torch
from transformers import AutoModelForCausalLM, BitsAndBytesConfig

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
    # llm_int8_skip_modules=["lm_head", "embed_tokens"] )
)

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map=device_map,
    torch_dtype="auto",
    use_cache=False, # set to False as we're going to use gradient checkpointing
    #flash_attn=True,
    #flash_rotary=True,
    #fused_dense=True,
    #low_cpu_mem_usage=True,
    trust_remote_code=True,
    #revision="refs/pr/23",
    #use_flash_attention_2=True, # Phi does not support yet.
    #attn_implementation="flash_attention_2", # set this to True if your GPU supports it (Flash Attention drastically speeds up model computations)
    #pretraining_tp=1 # 1 will activate the more accurate but slower computation of the linear layers, which should better match the original logits.
)


model.safetensors.index.json:   0%|          | 0.00/35.7k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/564M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [ ]:
# Used when we do not need to test the base model.

model_kwargs = dict(
    #attn_implementation="flash_attention_2", # set this to True if your GPU supports it (Flash Attention drastically speeds up model computations)
    torch_dtype="auto",
    use_cache=False, # set to False as we're going to use gradient checkpointing
    device_map=device_map,
    quantization_config=bnb_config,
)

In [ ]:
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

peft_config = LoraConfig(
    r=32, # higher for smaller models
    lora_alpha=64, # higher for smaller models
    target_modules= ["Wqkv", "fc1", "fc2" ], #["q_proj", "k_proj", "v_proj", "o_proj"], ["Wqkv", "fc1", "fc2" ] # ["Wqkv", "out_proj", "fc1", "fc2" ],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    #modules_to_save=["embed_tokens","lm_head"]
)

model = prepare_model_for_kbit_training(model, use_gradient_checkpointing=True)
model = get_peft_model(model, peft_config)

print_trainable_parameters(model)

trainable params: 26214400 || all params: 1547607040 || trainable%: 1.6938666807822222


Then we have to apply some preprocessing to the model to prepare it for training. For that use the `prepare_model_for_kbit_training` method from PEFT.

In [ ]:
# Model Architecture
print(model)

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): PhiForCausalLM(
      (model): PhiModel(
        (embed_tokens): Embedding(51200, 2560)
        (embed_dropout): Dropout(p=0.0, inplace=False)
        (layers): ModuleList(
          (0-31): 32 x PhiDecoderLayer(
            (self_attn): PhiAttention(
              (q_proj): Linear4bit(in_features=2560, out_features=2560, bias=True)
              (k_proj): Linear4bit(in_features=2560, out_features=2560, bias=True)
              (v_proj): Linear4bit(in_features=2560, out_features=2560, bias=True)
              (dense): Linear4bit(in_features=2560, out_features=2560, bias=True)
              (rotary_emb): PhiRotaryEmbedding()
            )
            (mlp): PhiMLP(
              (activation_fn): NewGELUActivation()
              (fc1): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=2560, out_features=10240, bias=True)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.0

# 4) Test Current Model Capability

In [ ]:
def generate_response(prompt, model):
    encoded_input = tokenizer(prompt,  return_tensors="pt", add_special_tokens=True)
    model_inputs = encoded_input.to('cuda')

    generated_ids = model.generate(**model_inputs,
                                 max_new_tokens=512,
                                 do_sample=True,
                                 pad_token_id=tokenizer.eos_token_id)

    decoded_output = tokenizer.batch_decode(generated_ids)

    return decoded_output[0].replace(prompt, "")

In [ ]:
prompt="What interesting things do you know abou love, technology and machine learninig?"

generate_response(prompt, model)

'\nYou might enjoy this podcast about new ways to use computer image recognition. Here’s a link to it on Spotify. Or go to it directly at this site.\nHave a great day!\nDr. John<|endoftext|>'

# 6) Setup Training Arguments

Let's load a common dataset, english quotes, to fine tune our model on famous quotes.

In [ ]:
# How many GPUs are in use ? -> paralell computing
if torch.cuda.device_count() > 1: # If more than 1 GPU
    print(torch.cuda.device_count())
    model.is_parallelizable = True
    model.model_parallel = True

Run the cell below to run the training! For the sake of the demo, we just ran it for few steps just to showcase how to use this integration with existing tools on the HF ecosystem.

In [ ]:
from transformers import TrainingArguments

args = TrainingArguments(
    output_dir = trained_lora,
    overwrite_output_dir=True,
    num_train_epochs=1,
    max_steps = 500, # comment out this line if you want to train in epochs
    gradient_accumulation_steps = 4, # batch size of 64 per_device_train_batch_size=4 and gradient_accumulation_steps=16 -> better use of the available GPU resources.
    per_device_train_batch_size = 1,
    #per_device_eval_batch_size = 1,
    gradient_checkpointing=True,
    gradient_checkpointing_kwargs={"use_reentrant": False},
    #do_eval=True,
    #evaluation_strategy="epoch",
    #evaluation_strategy="steps",
    #eval_steps=100, # comment out this line if you want to evaluate at the end of each epoch
    #warmup_steps = 0.03,
    warmup_ratio=0.05,
    weight_decay=0.01,
    save_steps=25,
    save_strategy="no",
    log_level="info",
    logging_steps=5,
    logging_strategy="steps",
    optim="paged_adamw_8bit",
    learning_rate=2.5e-5,
    lr_scheduler_type="cosine",
    fp16 = True, # specify bf16=True instead when training on GPUs that support bf16
    bf16 = False,
    tf32=False,
    # push_to_hub=True,
    # hub_model_id=new_model,
    # hub_strategy="every_save",
    # report_to="tensorboard",
    save_total_limit=None,
)


In [ ]:
from trl import SFTTrainer

max_seq_length = tokenizer.model_max_length

trainer = SFTTrainer(
  args=args,
  model=model,
  train_dataset=processed_dataset,
  #eval_dataset=processed_dataset["test"],
  dataset_text_field="text",
  peft_config=peft_config,
  max_seq_length=max_seq_length,
  tokenizer=tokenizer,
  packing=True,
  #formatting_func=create_prompt, # this will aplly the create_prompt mapping to all training and test dataset
)


Generating train split: 0 examples [00:00, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (15880 > 2048). Running this sequence through the model will result in indexing errors
max_steps is given, it will override any value given in num_train_epochs
Using auto half precision backend
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:318: UserWarning: You passed `packing=True` to the SFTTrainer, and you are training your model with `max_steps` strategy. The dataset will be iterated until the `max_steps` are reached.
  warnings.warn(


# 7) Train and Save

In [ ]:
train_result = trainer.train()

***** Running training *****
  Num examples = 4,092
  Num Epochs = 1
  Instantaneous batch size per device = 1
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 4
  Total optimization steps = 500
  Number of trainable parameters = 26,214,400


Step,Training Loss
5,2.791100
10,2.751300
15,2.815600
20,2.761200
25,2.778000
30,2.815700
35,2.781500
40,2.805100
45,2.779200
50,2.818900


Step,Training Loss
5,2.791100
10,2.751300
15,2.815600
20,2.761200
25,2.778000
30,2.815700
35,2.781500
40,2.805100
45,2.779200
50,2.818900


In [ ]:
metrics = train_result.metrics
max_train_samples = len(processed_dataset)
metrics["train_samples"] = min(max_train_samples, len(processed_dataset))
trainer.log_metrics("train", metrics)
trainer.save_metrics("train", metrics)
trainer.save_state()

In [ ]:
# model_to_save = trainer.model.module if hasattr(trainer.model, 'module') else trainer.model  # Take care of distributed/parallel training

trainer.save_model(trained_lora) # Saves weights

In [ ]:
#del model
#del pipe
#del trainer
#import gc
#gc.collect()
#gc.collect()

# 8) Save the LORA on the HUB

In [ ]:
# Save LoRA
trainer.push_to_hub(f"Teapack1/LoRA-{new_model}")

In [ ]:
# Load Model: (opt.1)
from transformers import AutoTokenizer, AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained(model_id)
lora_config = LoraConfig.from_pretrained(trained_lora)
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

In [ ]:
# Load Model: (opt.2)
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained(trained_lora)
model = AutoModelForCausalLM.from_pretrained(trained_lora, load_in_4bit=True, device_map='auto')

# 8) Merge LoRA with base model

In [ ]:
from peft import AutoPeftModelForCausalLM, PeftModel

base_model = AutoModelForCausalLM.from_pretrained(model_id,
                                             low_cpu_mem_usage=True,
                                             return_dict=True,
                                             torch_dtype=torch.float16,
                                             load_in_8bit=False,
                                             device_map=device_map,
                                             #trust_remote_code=True
                                                 )
peft_model = PeftModel.from_pretrained(
                                        base_model,
                                        trained_lora,
                                        from_transformers=True,
                                        device_map=device_map
                                        )

peft_model = peft_model.merge_and_unload()

"""
# Reload tokenizer to save it
tokenizer = AutoTokenizer.from_pretrained(
    model_id,
    #trust_remote_code=True
)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"
"""

# Save the merged model
peft_model.save_pretrained(new_model)
tokenizer.save_pretrained(new_model)

In [ ]:
peft_model.push_to_hub(f"Teapack1/merged-{new_model}")
tokenizer.push_to_hub(f"Teapack1/merged-{new_model}")

In [ ]:
# Define new model pro inference bellow
model = peft_model

# 9) Infere

In [ ]:
import torch

def generate_response(prompt, model, tokenizer):
    device = "cuda"

    #input_ids = tokenizer.apply_chat_template(prompt, truncation=True, add_generation_prompt=True, return_tensors="pt").to("device")

    input_ids = tokenizer(
        prompt,
        return_tensors="pt",
        add_special_tokens=True
    ).to(device)

    outputs = model.generate(
        **input_ids,
        max_new_tokens=120,
        temperature=0.5,
        top_k=50,
        top_p=0.95,
        repetition_penalty=1.2,
        penalty_alpha=0.6,
        do_sample = True,
        pad_token_id=tokenizer.eos_token_id
    )

    decoded_output = tokenizer.batch_decode(
        outputs,
        skip_special_tokens=True
    )[0]

    return decoded_output

In [ ]:
prompt = "[INST]Use the provided input to create an instruction that could have been used to generate the response with an LLM.\nThere are more than 12,000 species of grass. The most common is Kentucky Bluegrass, because it grows quickly, easily, and is soft to the touch. Rygrass is shiny and bright green colored. Fescues are dark green and shiny. Bermuda grass is harder but can grow in drier soil.[/INST]"

In [ ]:
print(generate_response(prompt, model, tokenizer))

##  Pipeline Inference

In [ ]:
from transformers import pipeline

gen = pipeline('text-generation', model=model, tokenizer=tokenizer, max_length=250)
result = gen(prompt)
print(result[0]['generated_text'].replace(prompt, ''))

# Generate Syntetic Dataset

In [ ]:
prompt = "A model that takes in a puzzle-like reasoning-heavy question in English, and responds with a well-reasoned, step-by-step thought out response in Spanish."
temperature = .4
number_of_examples = 100

In [ ]:
!pip install openai

In [ ]:
import os
import openai
import random

openai.api_key = "YOUR KEY HERE"

def generate_example(prompt, prev_examples, temperature=.5):
    messages=[
        {
            "role": "system",
            "content": f"You are generating data which will be used to train a machine learning model.\n\nYou will be given a high-level description of the model we want to train, and from that, you will generate data samples, each with a prompt/response pair.\n\nYou will do so in this format:\n```\nprompt\n-----------\n$prompt_goes_here\n-----------\n\nresponse\n-----------\n$response_goes_here\n-----------\n```\n\nOnly one prompt/response pair should be generated per turn.\n\nFor each turn, make the example slightly more complex than the last, while ensuring diversity.\n\nMake sure your samples are unique and diverse, yet high-quality and complex enough to train a well-performing model.\n\nHere is the type of model we want to train:\n`{prompt}`"
        }
    ]

    if len(prev_examples) > 0:
        if len(prev_examples) > 10:
            prev_examples = random.sample(prev_examples, 10)
        for example in prev_examples:
            messages.append({
                "role": "assistant",
                "content": example
            })

    response = openai.ChatCompletion.create(
        model="gpt-4",
        messages=messages,
        temperature=temperature,
        max_tokens=1354,
    )

    return response.choices[0].message['content']

# Generate examples
prev_examples = []
for i in range(number_of_examples):
    print(f'Generating example {i}')
    example = generate_example(prompt, prev_examples, temperature)
    prev_examples.append(example)

print(prev_examples)

In [ ]:
def generate_system_message(prompt):

    response = openai.ChatCompletion.create(
        model="gpt-4",
        messages=[
          {
            "role": "system",
            "content": "You will be given a high-level description of the model we are training, and from that, you will generate a simple system prompt for that model to use. Remember, you are not generating the system message for data generation -- you are generating the system message to use for inference. A good format to follow is `Given $INPUT_DATA, you will $WHAT_THE_MODEL_SHOULD_DO.`.\n\nMake it as concise as possible. Include nothing but the system prompt in your response.\n\nFor example, never write: `\"$SYSTEM_PROMPT_HERE\"`.\n\nIt should be like: `$SYSTEM_PROMPT_HERE`."
          },
          {
              "role": "user",
              "content": prompt.strip(),
          }
        ],
        temperature=temperature,
        max_tokens=500,
    )

    return response.choices[0].message['content']

system_message = generate_system_message(prompt)

print(f'The system message is: `{system_message}`. Feel free to re-run this cell if you want a better result.')

In [ ]:
import pandas as pd

# Initialize lists to store prompts and responses
prompts = []
responses = []

# Parse out prompts and responses from examples
for example in prev_examples:
  try:
    split_example = example.split('-----------')
    prompts.append(split_example[1].strip())
    responses.append(split_example[3].strip())
  except:
    pass

# Create a DataFrame
df = pd.DataFrame({
    'prompt': prompts,
    'response': responses
})

# Remove duplicates
df = df.drop_duplicates()

print('There are ' + str(len(df)) + ' successfully-generated examples. Here are the first few:')

df.head()

In [ ]:
# Split the data into train and test sets, with 90% in the train set
train_df = df.sample(frac=0.9, random_state=42)
test_df = df.drop(train_df.index)

# Save the dataframes to .jsonl files
train_df.to_json('train.jsonl', orient='records', lines=True)
test_df.to_json('test.jsonl', orient='records', lines=True)